In [1]:
import os
os.environ['NUMBA_DISABLE_JIT'] = "1"

from PySDM.physics import si
from PySDM.initialisation.sampling.spectral_sampling import ConstantMultiplicity
from PySDM.initialisation.spectra.exponential import Exponential

n_sd = 2 ** 15
initial_spectrum = Exponential(norm_factor=8.39e12, scale=1.19e5 * si.um ** 3)
attributes = {}
attributes['volume'], attributes['multiplicity'] = ConstantMultiplicity(initial_spectrum).sample(n_sd)

In [2]:
import numpy as np
from PySDM import Builder
from PySDM.environments import Box
from PySDM.dynamics import Coalescence
from PySDM.dynamics.collisions.collision_kernels import Golovin
from PySDM.backends import JAX, CPU
from PySDM.products import ParticleVolumeVersusRadiusLogarithmSpectrum

radius_bins_edges = np.logspace(np.log10(10 * si.um), np.log10(5e3 * si.um), num=32)

env = Box(dt=1 * si.s, dv=1e6 * si.m ** 3)
builder = Builder(n_sd=n_sd, backend=JAX(), environment=env)
builder.add_dynamic(Coalescence(collision_kernel=Golovin(b=1.5e3 / si.s), adaptive=False))
products = [ParticleVolumeVersusRadiusLogarithmSpectrum(radius_bins_edges=radius_bins_edges, name='dv/dlnr')]
particulator = builder.build(attributes, products)

print(particulator.attributes.keys())
print(particulator.attributes['volume'].data)

empty
get_empty_data: 0.051486968994140625
get_empty_data: 0.01377105712890625
empty
get_empty_data: 0.004712820053100586
get_empty_data: 7.104873657226562e-05
empty
get_empty_data: 0.014365196228027344
empty
get_empty_data: 7.319450378417969e-05
empty
get_empty_data: 0.007919073104858398
fill
fill: 5.0067901611328125e-06
empty
get_empty_data: 7.295608520507812e-05
fill
fill: 5.0067901611328125e-06
empty
get_empty_data: 0.011842012405395508
empty
get_empty_data: 6.699562072753906e-05
from_ndarray
get_data_from_ndarray
from_ndarray
get_data_from_ndarray
from_ndarray
get_data_from_ndarray
from_ndarray
get_data_from_ndarray
empty
get_empty_data: 0.012073993682861328
empty
get_empty_data: 5.817413330078125e-05
get_data_from_ndarray
empty
get_empty_data: 0.010919809341430664
empty
get_empty_data: 0.004091978073120117
empty
get_empty_data: 6.985664367675781e-05
rowview
upload
fill
fill: 5.9604644775390625e-06
empty
get_empty_data: 0.011930227279663086
upload
fill
fill: 5.0067901611328125e-06

In [3]:
from matplotlib import pyplot

for step in [0]: #, 1200, 2400, 3600]:
    particulator.run(step - particulator.n_steps)
    pyplot.step(
        x=radius_bins_edges[:-1] / si.um,
        y=particulator.formulae.particle_shape_and_density.volume_to_mass(
            particulator.products['dv/dlnr'].get()[0]
        ) / si.g,
        where='post', label=f"t = {step}s"
    )

pyplot.xscale('log')
pyplot.xlabel('particle radius [µm]')
pyplot.ylabel("dm/dlnr [g/m$^3$/(unit dr/r)]")
pyplot.legend()
pyplot.savefig('readme.png')

Attr name: volume
at


TypeError: JAX arrays are immutable and do not support in-place item assignment. Instead of x[idx] = y, use x = x.at[idx].set(y) or another .at[] method: https://docs.jax.dev/en/latest/_autosummary/jax.numpy.ndarray.at.html

In [ ]:
32768

31

31